# Pytorch and object-oriented programming

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.optim as optim
from torchmetrics import Accuracy

import torch.nn.init as init

## Pytorch Dataset

In [ ]:
class WaterDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        df = pd.read_csv(csv_path)
        self.data = df.to_numpy()
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        features = self.data[idx, :-1]
        label = self.data[idx, -1]
        return features, label

## Pytorch DataLoader

In [ ]:
dataset_train = WaterDataset("./data/water_train.csv")

dataloader_train = DataLoader(
    dataset_train,
    batch_size=2,
    shuffle=True,
)

features, labels = next(iter(dataloader_train))
print(features, labels)

## Pytorch Model

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(9, 16)
    self.fc2 = nn.Linear(16, 8)
    self.fc3 = nn.Linear(8, 1)
      
  def forward(self, x):
    x = nn.functional.relu(self.fc1(x))
    x = nn.functional.relu(self.fc2(x))
    x = nn.functional.sigmoid(self.fc3(x))
    return x

# Optimizers, training and evaluation

## Optimizers

In [ ]:
net = Net()

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [ ]:
optimizer = optim.RMSprop(net.parameters(), lr=0.001)

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.001)

## Model evaluation

In [ ]:
acc = Accuracy(task="binary")

net.eval()
with torch.no_grad():
    for features, labels in dataloader_test:
        outputs = net(features)
        preds = (outputs >= 0.5).float()
        acc(preds, labels.view(-1, 1))

test_accuracy = acc.compute()
print(f"Test accuracy: {test_accuracy}")

# Vanishing and exploiding gradients

## Initialization and activation

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(9, 16)
    self.fc2 = nn.Linear(16, 8)
    self.fc3 = nn.Linear(8, 1)
    
    init.kaiming_uniform_(self.fc1.weight)
    init.kaiming_uniform_(self.fc2.weight)
    init.kaiming_uniform_(
        self.fc3.weight,
        nonlinearity="sigmoid",)

  def forward(self, x):
    x = nn.functional.elu(self.fc1(x))
    x = nn.functional.elu(self.fc2(x))
    x = nn.functional.sigmoid(self.fc3(x))
    return x

## Batch normalization

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(9, 16)
    self.fc2 = nn.Linear(16, 8)
    self.fc3 = nn.Linear(8, 1)
    self.bn1 = nn.BatchNorm1d(16)
    self.bn2 = nn.BatchNorm1d(8)
    
    init.kaiming_uniform_(self.fc1.weight)
    init.kaiming_uniform_(self.fc2.weight)
    init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid") 
  
  def forward(self, x):
    x = self.fc1(x)
    x = self.bn1(x)
    x = nn.functional.elu(x)

    x = self.fc2(x)
    x = self.bn2(x)
    x = nn.functional.elu(x)

    x = nn.functional.sigmoid(self.fc3(x))
    return x